In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install timm

In [3]:
import sys
sys.path.append('drive/MyDrive/ai/final/petfinder-pawpularity-score')
### <디렉토리 설정하는 곳> (검색용) ################################################################################################
### 지금 위의 코드는
### 이 코드파일(ai_final_loss_plot.ipynb)과 함께 다른 데이터셋 관련파일들(test, train, sample_submission.csv, test.csv, train.csv)을 
### 구글드라이브 안의 /ai/final/petfinder-pawpularity-score 디렉토리에 넣어줘야, 코드가 돌아가도록 되어있다.
###
### 위에서 말한 코드파일과 데이터셋 관련 파일들의 위치를 바꾸고 싶으면, 
### 위의 sys.path.append() 안의 인자를 'drive/MyDrive/(구글 드라이브 내의 디렉토리 주소)' 로 바꿔주면 된다.
####################################################################################################################################

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch

import cv2
from os import path
from PIL import Image
import torch.utils.data as data
from torchvision import models, transforms
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import h5py
import timm

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from scipy.stats import pearsonr

In [5]:
class args:
### <배치크기, 에폭수 설정하는 곳 (검색용)> ######################
### 지금은 batch_size 값 32, 에폭수 10으로 되어있다. 
##################################################################

    seed = 22
    batch_size = 32
    epochs = 10
    image_size = 384  # scale shorter end of image to this size and centre crop
    central_fraction = 0.875  # only take this much of the centre when scaling and centre cropping
    load_img = False # whether load img or not
    workers = 2
    feature_extract = True

def seed_torch(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    print("seed: ", seed)

seed_torch(args.seed)

seed:  22


In [6]:
dir_path = 'drive/MyDrive/ai/final/petfinder-pawpularity-score'
### <디렉토리 설정하는 곳> (검색용) ################################################################################################
### 지금 위의 코드는
### 이 코드파일(ai_final_loss_plot.ipynb)과 함께 다른 데이터셋 관련파일들(test, train, sample_submission.csv, test.csv, train.csv)을 
### 구글드라이브 안의 /ai/final/petfinder-pawpularity-score 디렉토리에 넣어줘야, 코드가 돌아가도록 되어있다.
###
### 위에서 말한 코드파일과 데이터셋 관련 파일들의 위치를 바꾸고 싶으면, 
### 위의 sys.path.append() 안의 인자를 'drive/MyDrive/(구글 드라이브 내의 디렉토리 주소)' 로 바꿔주면 된다.
####################################################################################################################################

cached_dir = path.join(dir_path, 'cached_data')
train_img_dir = path.join(dir_path, 'train')
test_img_dir = path.join(dir_path, 'test')

df_train = pd.read_csv(path.join(dir_path, 'train.csv'))
df_test = pd.read_csv(path.join(dir_path, 'test.csv'))

train_img_paths = [path.join(train_img_dir, f"{img_id}.jpg") for img_id in df_train["Id"].values]
test_img_paths = [path.join(test_img_dir, f"{img_id}.jpg") for img_id in df_test["Id"].values]

In [7]:
property_names = [col for col in df_train.columns if col not in ['Id','Pawpularity']]
property_names # 12 properties

# train metda data
train_meta_X = df_train[property_names]
train_Y = df_train['Pawpularity']

# test metda data
test_id = df_test['Id']
test_meta_X = df_test.drop('Id',axis=1)
test_meta_X.shape

(8, 12)

In [8]:
class PawpularDataset(data.Dataset):
    def __init__(self, image_data, meta_features, labels, img_paths, augmentations=None):
        super(PawpularDataset, self).__init__()
        self.load_img = False
        self.image_data = image_data
        self.meta_features = meta_features
        self.labels = labels
        self.augmentations = augmentations
        self.image_paths = img_paths
        if self.augmentations is not None:
            self.load_img = True


    def __getitem__(self, item):
        if self.load_img:
            image = cv2.imread(self.image_paths[item])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            augmented = self.augmentations(image=image)["image"]
            image = np.transpose(augmented, (2, 0, 1)).astype(np.float32)
        else:
            image = self.image_data[item]

        meta = self.meta_features[item]
        label = self.labels[item]
        return image.astype('float32'), meta.astype('float32'), label

    def __len__(self):
        return len(self.labels)


def get_loader(split, features, meta_data, labels, img_paths=[], batch_size=args.batch_size, augmentations=None):
    """ Returns a data loader for the desired split """
    dataset = PawpularDataset(features, meta_data, labels, img_paths, augmentations)
    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True if split != 'test' else False,  # only shuffle the data in training
        pin_memory=True,
        num_workers=args.workers,
    )
    return loader

In [9]:
import albumentations

def get_augmentations(train=True):
    if train:
        return albumentations.Compose(
            [
                albumentations.Resize(args.image_size, args.image_size, p=1),
                albumentations.HorizontalFlip(p=0.5),
                albumentations.VerticalFlip(p=0.5),
                albumentations.Rotate(limit=180, p=0.7),
                albumentations.ShiftScaleRotate(
                    shift_limit = 0.1, scale_limit=0.1, rotate_limit=45, p=0.5
                ),
                albumentations.HueSaturationValue(
                    hue_shift_limit=0.2, sat_shift_limit=0.2,
                    val_shift_limit=0.2, p=0.5
                ),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=(-0.1, 0.1),
                    contrast_limit=(-0.1, 0.1), p=0.5
                ),
                albumentations.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                    p=1.0,
                ),
            ]
        )
    else:
        return albumentations.Compose(
            [
                albumentations.Resize(args.image_size, args.image_size, p=1),
                albumentations.Normalize(
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                    max_pixel_value=255.0,
                    p=1.0,
                ),
            ]
        )

augmentations = get_augmentations(train=True)
trainval_loader = get_loader('trainval', None, train_meta_X.values, train_Y.values.astype('float32'), train_img_paths,augmentations=augmentations)

augmentations = get_augmentations(train=False)
test_loader = get_loader('test', None, test_meta_X.values, torch.zeros(len(test_img_paths)), test_img_paths, augmentations=augmentations)

In [10]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

### <인공지능 모델 아키텍처 설정하는 곳> (검색용) ################################################################################################
### 아래에는 다음과 같은 모델 아키텍처를 정의하는 class 코드가 있음.
### 
### PretrainedCNN         - 이미 학습되어있는 "resnet" 모델을 불러오고, 마지막 layer에 metadata를 input data로 넣어주도록 고쳐준 모델
### BaseCNN               - CNN 을 활용한 모델, 마지막 layer에 metadata를 input data로 넣어주도록 고쳐준 모델
### BaseMLP               - MLP 를 활용한 모델, 마지막 layer에 metadata를 input data로 넣어주도록 고쳐준 모델
### PretrainedTransformer - 이미 학습되어있는 다양한 모델들 중 하나를 불러오고, 마지막 layer에 metadata를 input data로 넣어주도록 고쳐준 모델
###
### 이 아래에 class 중에서 직접 수정해보면 좋은 것은, BaseCNN , BaseMLP class 부분임.
### BaseCNN 은 CNN 실습했을때처럼 수정하면 될 듯
### BaseMLP 는 MLP 실습했을때처럼 수정하면 될 듯
###
### 한편, PretrainedTransformer class 를 활용해도 좋음. 이미 학습되어있는 모델을 불러와서 코드를 돌려볼 수 있음. 자세한 내용은 아래 <모델 고르기> 부분에서 설명함.
####################################################################################################################################



class PretrainedCNN(nn.Module):
    def __init__(self, model_name='resnet', use_meta=False):
        super(PretrainedCNN, self).__init__()
        if model_name == 'resnet':
            self.model = models.resnet152(pretrained=True)
            set_parameter_requires_grad(self.model, feature_extracting=True)
            num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 128)
        self.drop = nn.Dropout(0.5)
        self.use_meta =use_meta
        if self.use_meta:
            self.fc = nn.Linear(128+12, 1)
        else:
            self.fc = nn.Linear(128, 1)
        

    def forward(self, img_data, meta=None, targets=None):
        x = self.model(img_data)
        if self.use_meta:
            x = torch.cat([x, meta], dim=1)
        x = self.fc(self.drop(x)) # [b, o]
        return x



class BaseCNN(nn.Module):
    def __init__(self, num_filters=[], use_meta=False):
        super(BaseCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, num_filters[0], kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters[0], num_filters[1], kernel_size = 4, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(num_filters[1], num_filters[2], kernel_size = 5, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(num_filters[2] ,num_filters[3], kernel_size = 8, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(num_filters[3], num_filters[4], kernel_size = 6, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(num_filters[4], num_filters[4], kernel_size = 6, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Linear(12544, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,128)
        )
        # fc
        self.use_meta = use_meta
        self.drop = nn.Dropout(0.5)
        if self.use_meta:
            self.fc = nn.Linear(128+12, 1)
        else:
            self.fc = nn.Linear(128, 1)


    def forward(self, img_data, meta=None, targets=None):
        o = self.conv(self.drop(img_data)) # [b, 1, m]
        if self.use_meta:
            o = torch.cat([o, meta], dim=1)
        x = self.fc(self.drop(o)) # [b, o]
        return x


class BaseMLP(nn.Module):
    def __init__(self, use_meta=False, input_size=args.image_size):
        super(BaseMLP, self).__init__()
        # fc
        self.use_meta = use_meta
        self.drop = nn.Dropout(0.5)
        self.lin = nn.Sequential(
            nn.Linear(input_size*input_size, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(3),
            nn.Dropout(0.5),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(3),
            nn.Dropout(0.5),
            nn.Linear(512,128)
        )
        if self.use_meta:
            self.fc = nn.Linear((128*3)+12, 1)
        else:
            self.fc = nn.Linear(128*3, 1)


    def forward(self, img_data, meta=None, targets=None):
        b, c, img_size, _ = img_data.shape
        img_data = img_data.view(b, c, img_size*img_size)
        o = self.lin(self.drop(img_data)) # [b, 3, 128]
        o = o.view(b, -1)
        if self.use_meta:
            o = torch.cat([o, meta], dim=1)
        x = self.fc(self.drop(o)) # [b, o]
        return x


class PretrainedTransformer(nn.Module):
    def __init__(self, model_name='', use_meta=False, pretrained=True):
        super(PretrainedTransformer, self).__init__()
        # fc
        self.use_meta = use_meta
        self.drop = nn.Dropout(0.5)
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=0, in_chans=3)
        set_parameter_requires_grad(self.model, feature_extracting=True)
        num_features = self.model.num_features

        if self.use_meta:
            self.fc = nn.Linear(num_features+12, 1)
        else:
            self.fc = nn.Linear(num_features, 1)


    def forward(self, img_data, meta=None, targets=None):
        x = self.model(img_data)
        if self.use_meta:
            x = torch.cat([x, meta], dim=1)
        x = self.fc(self.drop(x)) # [b, o]
        return x




torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [11]:
import matplotlib.pyplot as plt ##5
test_loss_record = [] ##5
mse = nn.MSELoss()  ##


def train(model, optimizer, train_loader, val_loader, train=True, val=False, epoch=0, grad_clip=False):
    labels, preds = [], []
    if train:
        #mse = nn.MSELoss() -> 이건 밖으로 빼서 전역변수로 만들었음 ##
        train_loader = tqdm(train_loader, desc='{} E{:03d}'.format('train', epoch), ncols=0)
        model.train()
        for i, (img, meta_feature, label) in enumerate(train_loader):
            img = img.cuda().float()
            meta_feature = meta_feature.cuda()
            label = label.cuda()
            pred = model(img, meta_feature, label)

            # loss = criterion(pred , label.view(-1,1))
            loss = mse(pred, label.view(-1, 1))

            loss.backward()
            if grad_clip:
                nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            optimizer.zero_grad()

            num = 10
            if i % num == 0:
                rmse_loss = pow(loss.item() , 0.5)
                print("\tTrain Epoch:{} \tTrain Loss(RMSE): {:.6f}".format(epoch, rmse_loss)) ##
                test_loss_record.append(rmse_loss)
    
    if val:
        model.eval()
        val_loader = tqdm(val_loader, desc='{} E{:03d}'.format('val', epoch), ncols=0)
        for i, (img, meta_feature, label) in enumerate(val_loader):
            img = img.cuda().float()
            pred = model(img, meta_feature, None)

            labels.append(label.detach().cpu())
            preds.append(pred.detach().cpu())

        labels = torch.cat(labels, dim=0).numpy() # [num_seg]
        preds = torch.cat(preds, dim=0).numpy() # [num_seg]

    return labels, preds

In [12]:
### <학습된 모델 확인하기> (검색용) ######################
### 사용할 수 있는 이미 학습된 모델들을 아래 출력해줌
############################################################
timm.list_models()[:]

['adv_inception_v3',
 'bat_resnext26ts',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'deit_base_distilled_patch16_224',
 'deit_base_distilled_patch16_384',
 'deit_base_patch16_224',
 'deit_base_patch16_384',
 'deit_small_distilled_patch16_224',
 'deit_small_patch16_224',
 'deit_tiny_distilled_patch16_224',
 'deit_tiny_patch16_224',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2n

In [13]:
import torch.optim as optim

name = 'tf_efficientnet_b0_ns'

### <모델 고르기> (검색용) ##########################################################################################################
### 학습에 사용할 인공지능 모델을 골라준다.
### 아래 주석을 해제하여 사용
### 
### PretrainedTransformer 사용하는 방법
### 1. 아래 "model = PretrainedTransformer.." 가 있는 줄의 주석을 해제한다.
### 2. 아까 위에서 <학습된 모델 확인하기>에서 출력된 모델 중 하나를 고른다.
### 3. 아래 (학습된 모델 이름) 에 넣어준다.
#####################################################################################################################################
#model = PretrainedCNN(use_meta = True)                                          ### PretrainedCNN(resnet모델) 을 사용하는 경우
#model = BaseCNN(use_meta = True)                                                ### BaseCNN 을 사용하는 경우
#model = BaseMLP(use_meta = True)                                                ### BaseMLP 를 사용하는 경우
model = PretrainedTransformer(model_name='(학습된 모델 이름)', use_meta = True) ### PretrainedTransformer 을 사용하는 경우 


params_to_update = model.parameters()
print("Params to learn:")
if args.feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

### <optimizer 설정하는 곳> (검색용) ########################################################################
### 지금은 optim.Adam 으로 설정되어있음. optim.Adam 대신 다른 optimizer 알고리즘으로 바꿀수 있다.
###
### 다른 optimizer들 : optim.Adadelta , optim.Adagrad, optim.AdamW 등등
### 다른 optimizer로 바꾸려면, 해당 optimizer 알고리즘에 필요한 인자들을 알맞게 넣어줘야 한다. 
### 인자를 넣는 형식 등 자세한 사용법은 https://pytorch.org/docs/stable/optim.html 에 가서 
### Algorithms 부분 밑에 알고리즘을 클릭해서 들어가면 알 수 있음.
####################################################################################################
learning_rate = 1e-2
optimizer = optim.Adam(params_to_update, lr=learning_rate, amsgrad=False)
model = nn.DataParallel(model).cuda()

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

Params to learn:
	 model.fc.weight
	 model.fc.bias
	 fc.weight
	 fc.bias


In [14]:
##5  추가한 코드셀
''' 9. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    #correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += mse(output, label).item()
    
    test_loss /= (len(test_loader.dataset) / args.batch_size)

    return test_loss

In [ ]:
# # Train & val the model
best_score = 0
epochs = args.epochs



for epoch in range(epochs):
    labels, pred = train(model, optimizer, trainval_loader, None, train=True, val=False, epoch=epoch, grad_clip=True)

    torch.save({"model": model.state_dict()}, "efficient_model.pth")

train E000:   0% 1/310 [00:08<46:08,  8.96s/it]

	Train Epoch:0 	Train Loss(RMSE): 42.422983


train E000:   4% 11/310 [00:21<08:09,  1.64s/it]

	Train Epoch:0 	Train Loss(RMSE): 20.371416


train E000:   7% 21/310 [00:34<07:41,  1.60s/it]

...

train E003:  42% 131/310 [02:51<04:45,  1.59s/it]

	Train Epoch:3 	Train Loss(RMSE): 21.991761


train E003:  45% 141/310 [03:04<04:30,  1.60s/it]

	Train Epoch:3 	Train Loss(RMSE): 24.144286


train E003:  48% 149/310 [03:13<03:26,  1.28s/it]

In [ ]:
# Test the model
logs = torch.load("./efficient_model.pth")
model.load_state_dict(logs['model'])
_, pred = train(model, optimizer, None, test_loader, train=False, val=True, epoch=0, grad_clip=False)
print("Pawpularity score:{}".format(pred.mean().item()))

submission = pd.DataFrame({'Id':test_id,'Pawpularity':pred.squeeze(1)})

In [ ]:
submission

In [ ]:
##5
# test loss를 plotting 하기 위한 코드#########
plt.figure(figsize=(10,5))
plt.title("Test Loss")
plt.plot(test_loss_record, label="model 1")
plt.xlabel("batch")
plt.ylabel("Loss")
plt.legend()
plt.show()

### <RMSE 평균값 구하는 곳 > (검색용) ################################################################
### 지금은 RMSE 평균값 계산할때, RMSE 값들을 가장 마지막 "10개" 배치만을 이용하도록 되어있음
### 
### RMSE 평균값 계산할때, RMSE 값들을 가장 마지막 "n개" 배치만을 이용하도록 하고 싶으면,
### 아래 코드에서 "-10" 부분을, "-n" 으로 숫자를 바꿔주면 된다. 
#############################################################################################
last_batch_avg_RMSE = sum(test_loss_record[-10:] ,0.0) / len(test_loss_record[-10:])
print("\nlast {} batch average RMSE = {}".format(-(-10) ,last_batch_avg_RMSE))

In [ ]:
# <코드 수정 기록> 
# 2021.12.05(일) pm 3
# loss 출력을 RMSE로 바꿈
# 각 배치 마다 test loss 를 리스트로 저장해서, 학습완료후에 학습진행 그래프 출력되도록 코드 추가함
# 평균 RMSE를 계산하기 위해서, 학습중 마지막 10개 배치의 RMSE의 평균값을 출력하도록 코드 추가함
# 2021.12.05(일) pm 6
# 하이퍼파라미터튜닝 할 수 있도록 중요 코드 부분 주석처리함